In [ ]:
import requests
import pandas as pd
import time
from datetime import datetime, timedelta

requests.packages.urllib3.disable_warnings() 

# Base URL with placeholders for date and product
base_url = ("https://apim.misoenergy.org/pricing/v1/day-ahead/{date}/"
            "asm-expost?pageNumber=1&product={product}")

# Request headers with your subscription key
headers = {
    'Cache-Control': 'no-cache',
    'Ocp-Apim-Subscription-Key': 'your key',
}

# List of products to retrieve
products = ["Regulation", "Spin", "Supplemental","STR", "Ramp-up", "Ramp-down"]

# Define the date range for the year 2024 (2024 is a leap year)
start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 12, 31)

# Loop over each product one by one
for product in products:
    print(f"Downloading data for {product}...")
    product_data = []  # List to store data for the current product
    
    # Loop through every day in 2024 for the current product
    current_date = start_date
    while current_date <= end_date:
        date_str = current_date.strftime("%Y-%m-%d")
        # Format the URL with the current date and product
        url = base_url.format(date=date_str, product=product)
        
        # Make the GET request
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            json_response = response.json()
            for entry in json_response.get("data", []):
                # Extract the "end" value from the nested timeInterval dictionary
                if "timeInterval" in entry and "end" in entry["timeInterval"]:
                    entry["end"] = entry["timeInterval"]["end"]
                else:
                    entry["end"] = None
                
                # Add the query date to the record for reference
                entry["query_date"] = date_str
                
                # Append the record to the product-specific list
                product_data.append(entry)
        else:
            print(f"Error {response.status_code} for {date_str} and product {product}")
        
        # Delay to ensure we remain below 100 API calls per minute
        # time.sleep(0.7)
        current_date += timedelta(days=1)
    
    # Convert the collected data for the product into a DataFrame
    df_product = pd.DataFrame(product_data)
    
    # Optionally convert the 'end' column to datetime for proper handling
    df_product['end'] = pd.to_datetime(df_product['end'])
    
    # Pivot the DataFrame so that the index is the "end" time and the columns are zones (with mcp as the values)
    pivot_table = df_product.pivot_table(index='end', columns='zone', values='mcp', aggfunc='first')
    
    # Save the pivoted data for the current product to a CSV file
    filename = f"2024_{product}_data.csv"
    pivot_table.to_csv(filename)
    print(f"Data for {product} saved to {filename}")


In [ ]:
import pandas as pd

# Read the CSV file, ensuring the 'end' column is parsed as datetime and set as the index.
for products in ["Regulation", "Spin", "Supplemental", "STR", "Ramp-up", "Ramp-down"]:

    print(f'checking {products} file')

    filename = f"2024_{products}_data.csv"  # Replace with your file name if needed
    df = pd.read_csv(filename, parse_dates=['end'], index_col='end')

    # Ensure the index is sorted
    df.sort_index(inplace=True)

    # Create a complete hourly date range from the earliest to the latest 'end' timestamp
    full_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='H')

    # Identify missing hours by checking which expected timestamps are not in the dataframe index
    missing_hours = full_range.difference(df.index)

    print("Missing Hours:")
    print(missing_hours)

    # Identify duplicate hours by checking if any index value appears more than once
    duplicates = df.index[df.index.duplicated(keep=False)].unique()

    print("\nDuplicate Hours:")
    print(duplicates)

    # Optionally, display detailed rows for the duplicate hours
    duplicate_details = df[df.index.duplicated(keep=False)]
    print("\nDetails of Duplicate Rows:")
    print(duplicate_details)


In [ ]:
from gridstatusio import GridStatusClient

client = GridStatusClient(api_key="your key")

name = "pjm_as_market_results_real_time"

df = client.get_dataset(
    dataset=name,
    start="2024-01-01 00:00",
    end="2025-01-01 00:00",
    timezone="US/Eastern",
    filter_column="location",
    filter_value="MHK VL"
)
df.to_csv(f"{name}_2024.csv", index=False)

In [ ]:
import pandas as pd

def process_iso_data(config):
    """
    Processes ancillary service price data for a given ISO based on configuration.

    Args:
        config (dict): A dictionary containing ISO-specific configuration:
            iso_name (str): Name of the ISO (for messages).
            filename (str): Path to the input CSV or Excel file.
            file_type (str): 'csv' or 'excel' (optional, detected from extension if None).
            timezone (str): IANA timezone name (e.g., 'US/Central').
            time_col (str): Name of the source local time column.
            data_format (str): 'wide' or 'long'.
            filter_col (str | None): Column name for filtering, or None.
            filter_val (str | None): Value to filter for in filter_col, or None.
            avg_zones (bool): If True and format is 'wide', average across all zones.
            zone_col (str | None): Name of the zone column.
            resample_freq (str | None): Pandas frequency string for resampling (e.g., 'H'), or None.
            resample_agg (str | None): Aggregation method for resampling ('mean', 'sum', etc.), or None.
            value_cols (list): List of original columns needed for processing.
            service_type_col_long (str | None): Col name for service type ('long' format).
            value_col_long (str | None): Col name for numerical value ('long' format).
            rename_map (dict): Dictionary mapping original AS names/types to abbreviations.

    Returns:
        pandas.DataFrame: Processed DataFrame or None if processing fails.
    """
    iso_name = config['iso_name']
    filename = config['filename']
    file_type = config.get('file_type') # Optional config parameter

    print(f"--- Processing {iso_name} data from {filename} ---")

    try:
        # --- Load Data ---
        if file_type == 'excel' or filename.endswith(('.xlsx', '.xls')):
            print(f"Reading Excel file: {filename}")
            df = pd.read_excel(filename)
        elif file_type == 'csv' or filename.endswith('.csv'):
             print(f"Reading CSV file: {filename}")
             df = pd.read_csv(filename)
        else:
            # Default to CSV if type not specified and extension unknown
             print(f"Assuming CSV format for: {filename}")
             df = pd.read_csv(filename)


        # --- Initial Filtering ---
        if config['filter_col'] and config['filter_val']:
            print(f"Filtering for {config['filter_col']} == '{config['filter_val']}'")
            if config['filter_col'] not in df.columns:
                 raise ValueError(f"Filter column '{config['filter_col']}' not found.")
            df_filtered = df[df[config['filter_col']] == config['filter_val']].copy()
            if df_filtered.empty:
                print(f"Warning: No rows found after filtering for {config['filter_val']}. Skipping {iso_name}.")
                return None
        else:
            df_filtered = df.copy()

        # --- Select Necessary Columns ---
        required_cols = [config['time_col']] + config['value_cols']
        if config['avg_zones'] and config['zone_col'] and config['zone_col'] not in required_cols:
             required_cols.append(config['zone_col'])

        missing_cols = [col for col in required_cols if col not in df_filtered.columns]
        if missing_cols:
            raise ValueError(f"Missing required columns: {missing_cols}")
        df_selected = df_filtered[required_cols].copy()

        # --- Time Processing ---
        print(f"Processing time column '{config['time_col']}' to timezone '{config['timezone']}'")
        df_selected['Time_parsed'] = pd.to_datetime(df_selected[config['time_col']], errors='coerce')
        nat_count = df_selected['Time_parsed'].isna().sum()
        df_selected.dropna(subset=['Time_parsed'], inplace=True)
        if df_selected.empty:
            raise ValueError("DataFrame empty after time conversion and dropna.")

        parsed_dtype = df_selected['Time_parsed'].dtype
        processed_time_col = 'Time'

        if pd.api.types.is_object_dtype(parsed_dtype) and nat_count == 0:
            time_utc_series = pd.to_datetime(df_selected['Time_parsed'], utc=True)
            time_target_series = time_utc_series.dt.tz_convert(config['timezone'])
            df_selected[processed_time_col] = time_target_series
        elif pd.api.types.is_datetime64_any_dtype(parsed_dtype):
            time_series = df_selected['Time_parsed']
            if time_series.dt.tz is None:
                df_selected[processed_time_col] = time_series.dt.tz_localize(config['timezone'], ambiguous='infer', nonexistent='shift_forward')
            else:
                df_selected[processed_time_col] = time_series.dt.tz_convert(config['timezone'])
        else:
            raise TypeError(f"Unexpected dtype '{parsed_dtype}' for parsed time column.")

        # Prepare columns for next step
        cols_for_processing = [processed_time_col] + config['value_cols']
        # Remove time_col if it's same as processed_time_col to avoid duplicate column issues later
        if config['time_col'] in cols_for_processing and config['time_col']!= processed_time_col:
             pass # keep both if names different
        elif config['time_col'] == processed_time_col:
             cols_for_processing.remove(config['time_col']) # Avoid duplicate time columns if original name was 'Time'

        df_processed_time = df_selected[list(set(cols_for_processing))].copy() # Use set to ensure unique columns
        print(f"Time processing successful. Dtype: {df_processed_time[processed_time_col].dtype}")

        # --- Data Structuring (Pivot/Average/Select) ---
        # Set Time as index early for potential resampling/groupby
        if processed_time_col in df_processed_time.columns:
             df_processed_time.set_index(processed_time_col, inplace=True)
        elif isinstance(df_processed_time.index, pd.DatetimeIndex):
             pass # Already indexed correctly
        else:
             raise ValueError("Cannot find or set time index before structuring.")

        value_cols_only = [col for col in config['value_cols'] if col != config.get('service_type_col_long')] # Get only numeric value columns

        if config['data_format'] == 'long':
            print("Pivoting data from long to wide format...")
            # Need service_type and value columns from original selection
            pivot_df = df_selected[[processed_time_col, config['service_type_col_long'], config['value_col_long']]].copy()
            pivot_df.set_index(processed_time_col, inplace=True)

            df_pivoted = pivot_df.pivot_table(
                index=pivot_df.index, # Use existing datetime index
                columns=config['service_type_col_long'],
                values=config['value_col_long'],
                aggfunc='first'
            )
            df_pivoted.columns.name = None
            df_structured = df_pivoted
            # Rename using keys from rename_map
            df_structured = df_structured.rename(columns=config['rename_map'])

        elif config['data_format'] == 'wide':
            if config['avg_zones']:
                print("Averaging values across zones...")
                # Group by time index and average the specified value columns
                df_averaged = df_processed_time[value_cols_only].groupby(df_processed_time.index).mean()
                df_structured = df_averaged
                # Rename happens AFTER averaging
                df_structured = df_structured.rename(columns=config['rename_map'])
            else:
                 print("Selecting and renaming columns for wide format (no averaging)...")
                 # Just select the value columns and rename
                 df_structured = df_processed_time[value_cols_only].rename(columns=config['rename_map'])
        else:
             raise ValueError(f"Unknown data_format: {config['data_format']}")

        # --- Optional Resampling (e.g., for ISONE 5-min to hourly) ---
        if config.get('resample_freq') and config.get('resample_agg'):
            freq = config['resample_freq']
            agg_method = config['resample_agg']
            print(f"Resampling data to frequency '{freq}' using '{agg_method}'...")
            if agg_method == 'mean':
                 df_structured = df_structured.resample(freq).mean()
            elif agg_method == 'sum':
                 df_structured = df_structured.resample(freq).sum()
            # Add other aggregation methods if needed
            else:
                 print(f"Warning: Unsupported resample aggregation '{agg_method}'. Using 'mean'.")
                 df_structured = df_structured.resample(freq).mean()


        # Ensure columns match target names after structuring/renaming/resampling
        final_col_names = list(config['rename_map'].values())
        missing_final_cols = [col for col in final_col_names if col not in df_structured.columns]
        if missing_final_cols:
             print(f"Warning: Columns {missing_final_cols} not found after processing/renaming.")
             print(f"Available columns: {df_structured.columns.tolist()}")
             final_col_names = [col for col in final_col_names if col in df_structured.columns]
             if not final_col_names:
                  raise ValueError("No target columns found after processing.")

        df_renamed_final = df_structured[final_col_names] # Keep only successfully processed columns

        # Check index again before reindexing
        if not isinstance(df_renamed_final.index, pd.DatetimeIndex):
             raise TypeError("Index is not a DatetimeIndex before reindexing.")
        print(f"Data structured. Index timezone: {df_renamed_final.index.tz}")

        # --- Reindex, Fillna, Filter ---
        print("Reindexing to full 2024 hourly range, filling NaNs, removing Feb 29...")
        # Always reindex to hourly for final comparison, using the target timezone
        full_range = pd.date_range(start='2024-01-01 00:00:00', end='2024-12-31 23:00:00', freq='H', tz=config['timezone'])

        # Handle duplicates (safer, though less likely after resample/groupby)
        df_renamed_final = df_renamed_final[~df_renamed_final.index.duplicated(keep='first')]

        # Calculate column means from the structured data *before* reindexing
        column_means = df_renamed_final[final_col_names].mean()

        # Reindex
        df_reindexed = df_renamed_final.reindex(full_range)

        # Fill NaNs
        df_filled = df_reindexed.fillna(column_means)

        # Remove Feb 29
        mask = ~((df_filled.index.month == 2) & (df_filled.index.day == 29))
        df_final_iso = df_filled[mask].copy()

        # --- Final Formatting ---
        df_final_iso.reset_index(inplace=True)
        df_final_iso = df_final_iso.rename(columns={'index': 'Time'})
        df_final_iso['Time'] = df_final_iso['Time'].dt.tz_localize(None) # Make naive

        # Round numeric columns to 3 decimal places
        numeric_cols_final = df_final_iso.select_dtypes(include=['number']).columns
        df_final_iso[numeric_cols_final] = df_final_iso[numeric_cols_final].round(3)

        df_final_iso = df_final_iso[['Time'] + final_col_names] # Ensure final column order

        print(f"--- {iso_name} processing complete. Final shape: {df_final_iso.shape} ---")
        return df_final_iso

    except FileNotFoundError:
        print(f"Error: File not found at {filename}. Skipping {iso_name}.")
        return None
    except Exception as e:
        print(f"Error processing {iso_name} data: {e}")
        import traceback
        traceback.print_exc() # Print detailed traceback for debugging
        return None

# --- Configuration Dictionaries (Including ISONE) ---

spp_config = {
    'iso_name': 'SPP',
    'filename': 'spp_as_prices_2024.csv',
    'file_type': 'csv',
    'timezone': 'US/Central',
    'time_col': 'interval_start_local',
    'data_format': 'wide',
    'filter_col': 'reserve_zone',
    'filter_val': 'SPP',
    'avg_zones': False, 'zone_col': None,
    'resample_freq': None, 'resample_agg': None,
    'value_cols': ['reg_up', 'reg_dn', 'ramp_up', 'ramp_dn', 'spin', 'supp', 'unc_up'],
    'service_type_col_long': None, 'value_col_long': None,
    'rename_map': {'reg_up': 'RegU', 'reg_dn': 'RegD', 'ramp_up': 'RamU', 'ramp_dn': 'RamD', 'spin': 'Spin', 'supp': 'Sup', 'unc_up': 'UncU'}
}

caiso_config = {
    'iso_name': 'CAISO',
    'filename': 'caiso_as_prices_2024.csv',
    'file_type': 'csv',
    'timezone': 'US/Pacific',
    'time_col': 'interval_start_local',
    'data_format': 'wide',
    'filter_col': 'region', 'filter_val': 'AS_CAISO', # Single region version
    'avg_zones': False, 'zone_col': 'region',
    'resample_freq': None, 'resample_agg': None,
    'value_cols': ['regulation_up', 'regulation_down', 'spinning_reserves', 'non_spinning_reserves', 'regulation_mileage_up', 'regulation_mileage_down'],
    'service_type_col_long': None, 'value_col_long': None,
    'rename_map': {'regulation_up': 'RegU', 'regulation_down': 'RegD', 'spinning_reserves': 'Spin', 'non_spinning_reserves': 'NSpin', 'regulation_mileage_up': 'RMU', 'regulation_mileage_down': 'RMD'}
}

ercot_config = {
    'iso_name': 'ERCOT',
    'filename': 'ercot_as_prices_2024.csv',
    'file_type': 'csv',
    'timezone': 'US/Central',
    'time_col': 'interval_start_local',
    'data_format': 'wide',
    'filter_col': None, 'filter_val': None,
    'avg_zones': False, 'zone_col': None,
    'resample_freq': None, 'resample_agg': None,
    'value_cols': ['regulation_up', 'regulation_down', 'responsive_reserves', 'non_spinning_reserves', 'ercot_contingency_reserve_service'],
    'service_type_col_long': None, 'value_col_long': None,
    'rename_map': {'regulation_up': 'RegU', 'regulation_down': 'RegD', 'responsive_reserves': 'Spin', 'non_spinning_reserves': 'NSpin', 'ercot_contingency_reserve_service': 'ECRS'}
}

pjm_config = {
    'iso_name': 'PJM',
    'filename': 'pjm_as_market_results_real_time_2024.csv',
    'file_type': 'csv', # Assuming CSV despite user code saving to CSV
    'timezone': 'US/Eastern',
    'time_col': 'interval_start_local',
    'data_format': 'long',
    'filter_col': 'locale', 'filter_val': 'PJM RTO Reserve Zone',
    'avg_zones': False, 'zone_col': 'locale',
    'resample_freq': None, 'resample_agg': None, # Assume source is hourly after pivot
    'value_cols': ['market_clearing_price','service_type'],
    'service_type_col_long': 'service_type', 'value_col_long': 'market_clearing_price',
    'rename_map': {'Primary Reserve': 'Rse', 'Regulation': 'Reg', 'Synchronized Reserve': 'Syn', 'Thirty Minutes Reserve': 'TMR'}
}

nyiso_config = {
    'iso_name': 'NYISO',
    'filename': 'nyiso_as_prices_day_ahead_hourly_2024.csv',
    'file_type': 'csv',
    'timezone': 'US/Eastern',
    'time_col': 'interval_start_local',
    'data_format': 'wide',
    'filter_col': None, 'filter_val': None,
    'avg_zones': True, # Average across zones
    'zone_col': 'zone',
    'resample_freq': None, 'resample_agg': None, # Already hourly
    'value_cols': ['spin_reserves_10_min', 'non_spin_reserves_10_min', 'reserves_30_min', 'regulation_capacity'],
    'service_type_col_long': None, 'value_col_long': None,
    'rename_map': {'spin_reserves_10_min': 'Spin10', 'non_spin_reserves_10_min': 'NSpin10', 'reserves_30_min': 'Res30', 'regulation_capacity': 'RegC'}
}

isone_config = {
    'iso_name': 'ISONE',
    'filename': '5min_reserve_price_and_designation_2024.xlsx', # Excel file
    'file_type': 'excel', # Specify excel type
    'timezone': 'US/Eastern',
    'time_col': 'local_time', # From user code
    'data_format': 'wide',
    'filter_col': None, 'filter_val': None, # No filtering mentioned
    'avg_zones': False, 'zone_col': None, # Assumed system wide
    'resample_freq': 'H', # Resample from 5min to Hourly
    'resample_agg': 'mean', # Aggregate using mean
    'value_cols': ['tmsr_dollar_per_MWh', 'tmnsr_dollar_per_MWh', 'tmor_dollar_per_MWh'], # From user code
    'service_type_col_long': None, 'value_col_long': None,
    'rename_map': {'tmsr_dollar_per_MWh': 'Spin10', 'tmnsr_dollar_per_MWh': 'NSpin10', 'tmor_dollar_per_MWh': 'OR30'} # New abbreviations
}

# --- Example Usage ---
# You can uncomment and run the processing for any ISO you have the file for.
# Make sure the filenames in the config match the actual files available.

# df_spp_final = process_iso_data(spp_config)
# df_caiso_final = process_iso_data(caiso_config)
# df_ercot_final = process_iso_data(ercot_config)
# df_pjm_final = process_iso_data(pjm_config)
# df_nyiso_final = process_iso_data(nyiso_config)

# Process ISONE data
print("\nProcessing ISONE data...")
df_isone_final = process_iso_data(isone_config)

if df_isone_final is not None:
    print("\nISONE Final Head:\n", df_isone_final.head().to_markdown(index=False))
    # Save the ISONE results as specified in the user's code snippet
    output_filename = "isone_as_prices_2024.csv"
    df_isone_final.to_csv(output_filename, index=False)
    print(f"\nISONE processed data saved to '{output_filename}'")

In [1]:
iso_service_abbreviations = {
    'SPP': [
        'RegU',    # Regulation Up
        'RegD',    # Regulation Down
        'RamU',    # Ramp Up
        'RamD',    # Ramp Down
        'Spin',    # Spinning Reserve
        'Sup',     # Supplemental Reserve
        'UncU'     # Uncertainty Up (may relate to ramp/reserves)
    ],
    'CAISO': [
        'RegU',    # Regulation Up
        'RegD',    # Regulation Down
        'Spin',    # Spinning Reserves
        'NSpin',   # Non-Spinning Reserves
        'RMU',     # Ramp Up
        'RMD'      # Ramp Down
    ],
    'ERCOT': [
        'RegU',    # Regulation Up
        'RegD',    # Regulation Down
        'Spin',    # Responsive Reserves (Mapped to Spin)
        'NSpin',   # Non-Spinning Reserves
        'ECRS'     # ERCOT Contingency Reserve Service
    ],
    'PJM': [
        'Rse',     # Primary Reserve
        'Reg',     # Regulation
        'Syn',     # Synchronized Reserve
        'TMR'      # Thirty Minutes Reserve
    ],
    'NYISO': [
        'Spin10',  # 10-Minute Spinning Reserves
        'NSpin10', # 10-Minute Non-Spinning Reserves
        'Res30',   # 30-Minute Reserves
        'RegC'     # Regulation Capacity
    ],
    'ISONE': [
        'Spin10',  # Ten Minute Spinning Reserve (TMSR)
        'NSpin10', # Ten Minute Non-Spinning Reserve (TMNSR)
        'OR30'     # Thirty Minute Operating Reserve (TMOR)
    ],
    'MISO': [
        'RamU',    # Ramp Up
        'RamD',    # Ramp Down
        'Spin',    # Spinning Reserve
        'STR',     # Short-term reserve
        'Sup',     # Supplemental Reserve
        'Reg'      # Regulation
    ]
}


In [ ]:
import os
import pandas as pd
import glob
from datetime import datetime
import logging

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('price_data_transfer.log'),
        logging.StreamHandler()
    ]
)


# Special mappings for certain ISOs
special_mappings = {
    'PJM': {'Reg': ['RegUp', 'RegDown']},
    'MISO': {'Reg': ['RegUp', 'RegDown']},
    'NYISO': {'RegC': ['RegUp', 'RegDown']}
}

def process_iso_files():
    """
    Process all ISO price files and update the corresponding hourly data files
    """
    # Source directory for ISO AS price files
    iso_dir = './ISOs'
    # Target directory containing ISO folders with hourly data files
    target_base_dir = '../input/hourly_data'
    
    # Get all AS price files
    iso_files = glob.glob(os.path.join(iso_dir, '*_as_prices_2024.csv'))
    
    if not iso_files:
        logging.error(f"No ISO AS price files found in {iso_dir}")
        return
    
    for iso_file in iso_files:
        # Extract ISO name from filename
        file_name = os.path.basename(iso_file)
        iso_name = file_name.split('_')[0].upper()
        
        logging.info(f"Processing {iso_name} from file {file_name}")
        
        try:
            # Read the ISO AS price data
            iso_data = pd.read_csv(iso_file)
            
            # Get the target file path
            target_dir = os.path.join(target_base_dir, iso_name)
            target_file = os.path.join(target_dir, 'Price_ANS_hourly.csv')
            
            if not os.path.exists(target_file):
                logging.error(f"Target file not found: {target_file}")
                continue
            
            # Read the target hourly data file
            target_data = pd.read_csv(target_file)
            
            # Process and update the target data
            target_data = update_target_data(iso_name, iso_data, target_data)
            
            # Save the updated data
            target_data.to_csv(target_file, index=False)
            logging.info(f"Updated {target_file} successfully")
            
        except Exception as e:
            logging.error(f"Error processing {iso_name}: {str(e)}")

def update_target_data(iso_name, source_data, target_data):
    """
    Update target data with prices from source data, matching timestamps while ignoring year differences
    
    Args:
        iso_name: Name of the ISO
        source_data: DataFrame containing the source price data (2024)
        target_data: DataFrame to be updated (2023)
        
    Returns:
        Updated target DataFrame
    """
    # Get the first column name (timestamp column)
    time_col = source_data.columns[0]
    target_time_col = target_data.columns[0]
    
    # Convert timestamps to datetime objects
    try:
        if pd.api.types.is_object_dtype(source_data[time_col]):
            source_data[time_col] = pd.to_datetime(source_data[time_col])
        if pd.api.types.is_object_dtype(target_data[target_time_col]):
            target_data[target_time_col] = pd.to_datetime(target_data[target_time_col])
    except Exception as e:
        logging.warning(f"Timestamp conversion issue for {iso_name}: {str(e)}")
    
    # Create matching keys based on month, day, and hour (ignoring year)
    # This allows 2023 data to match with 2024 data
    source_data['match_key'] = source_data[time_col].dt.strftime('%m-%d-%H')
    target_data['match_key'] = target_data[target_time_col].dt.strftime('%m-%d-%H')
    
    # Set all 'loc_' columns to 0
    loc_columns = [col for col in target_data.columns if col.startswith('loc_')]
    for col in loc_columns:
        logging.info(f"Setting column {col} to 0 for {iso_name}")
        target_data[col] = 0
    
    # Get services available for this ISO
    available_services = [col for col in source_data.columns if col != time_col and col != 'match_key']
    
    for service in available_services:
        if service in iso_service_abbreviations.get(iso_name, []):
            # Handle special mappings for PJM, MISO, and NYISO
            if iso_name in special_mappings and service in special_mappings[iso_name]:
                target_services = special_mappings[iso_name][service]
                for target_service in target_services:
                    target_col = f"p_{target_service}_{iso_name}"
                    if target_col in target_data.columns:
                        logging.info(f"Mapping {service} to {target_col} for {iso_name}")
                        update_column_by_match_key(source_data, target_data, service, target_col)
                    else:
                        logging.warning(f"Target column {target_col} not found in target data for {iso_name}")
            else:
                # Standard mapping
                target_col = f"p_{service}_{iso_name}"
                if target_col in target_data.columns:
                    logging.info(f"Updating column {target_col} for {iso_name}")
                    update_column_by_match_key(source_data, target_data, service, target_col)
                else:
                    logging.warning(f"Target column {target_col} not found in target data for {iso_name}")
    
    # Clean up the temporary match_key column
    if 'match_key' in target_data.columns:
        target_data.drop('match_key', axis=1, inplace=True)
    
    return target_data

def update_column_by_match_key(source_data, target_data, source_col, target_col):
    """
    Update a specific column in the target DataFrame using match_key (month-day-hour)
    
    Args:
        source_data: Source DataFrame
        target_data: Target DataFrame to update
        source_col: Column name in source data
        target_col: Column name in target data
    """
    # Create a mapping dictionary from match_key to values
    source_map = dict(zip(source_data['match_key'], source_data[source_col]))
    
    # Update the target column based on matching keys
    for idx, row in target_data.iterrows():
        match_key = row['match_key']
        if match_key in source_map:
            target_data.at[idx, target_col] = source_map[match_key]

if __name__ == "__main__":
    logging.info("Starting ISO price data transfer process")
    process_iso_files()
    logging.info("ISO price data transfer process completed")

In [ ]:
import os
import pandas as pd
import glob
import logging
from datetime import datetime

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('lmp_data_transfer.log'),
        logging.StreamHandler()
    ]
)

def process_lmp_data():
    """
    Process LMP data for all ISOs, calculate average across hubs, and update target files
    """
    # Source directory for ISO LMP files
    lmp_dir = './Raw/LMP'
    # Target directory containing ISO Price_hourly.csv files
    target_base_dir = '../input/hourly_data'
    
    # Get all LMP files
    lmp_files = glob.glob(os.path.join(lmp_dir, '*_CLEANED.csv'))
    
    if not lmp_files:
        logging.error(f"No ISO LMP files found in {lmp_dir}")
        return
    
    for lmp_file in lmp_files:
        # Extract ISO name from filename
        file_name = os.path.basename(lmp_file)
        iso_name = file_name.split('_')[0].upper()
        
        logging.info(f"Processing LMP data for {iso_name} from file {file_name}")
        
        try:
            # Read the ISO LMP data
            lmp_data = pd.read_csv(lmp_file)
            
            # Get the target file path
            target_dir = os.path.join(target_base_dir, iso_name)
            target_file = os.path.join(target_dir, 'Price_hourly.csv')
            
            if not os.path.exists(target_file):
                logging.error(f"Target file not found: {target_file}")
                continue
            
            # Read the target hourly data file
            target_data = pd.read_csv(target_file)
            
            # Process LMP data and update target file
            target_data = update_price_data(iso_name, lmp_data, target_data)
            
            # Save the updated data
            target_data.to_csv(target_file, index=False)
            logging.info(f"Updated {target_file} successfully with average LMP data")
            
        except Exception as e:
            logging.error(f"Error processing {iso_name} LMP data: {str(e)}")

def update_price_data(iso_name, lmp_data, target_data):
    """
    Calculate average LMP across hubs and update target data
    
    Args:
        iso_name: Name of the ISO
        lmp_data: DataFrame containing the LMP data (2024)
        target_data: Target DataFrame to be updated (2023)
        
    Returns:
        Updated target DataFrame
    """
    # Get the first column name (timestamp column) from both dataframes
    lmp_time_col = lmp_data.columns[0]
    target_time_col = target_data.columns[0]
    
    # Convert timestamps to datetime objects
    try:
        if pd.api.types.is_object_dtype(lmp_data[lmp_time_col]):
            lmp_data[lmp_time_col] = pd.to_datetime(lmp_data[lmp_time_col])
        if pd.api.types.is_object_dtype(target_data[target_time_col]):
            target_data[target_time_col] = pd.to_datetime(target_data[target_time_col])
    except Exception as e:
        logging.warning(f"Timestamp conversion issue for {iso_name}: {str(e)}")
    
    # Create matching keys based on month, day, and hour (ignoring year)
    lmp_data['match_key'] = lmp_data[lmp_time_col].dt.strftime('%m-%d-%H')
    target_data['match_key'] = target_data[target_time_col].dt.strftime('%m-%d-%H')
    
    # Get all hub columns (exclude the timestamp column)
    hub_columns = [col for col in lmp_data.columns if col != lmp_time_col and col != 'match_key']
    
    if not hub_columns:
        logging.warning(f"No hub columns found in LMP data for {iso_name}")
        return target_data
    
    # Calculate average LMP across all hubs for each timestamp
    lmp_data['avg_lmp'] = lmp_data[hub_columns].mean(axis=1).round(3)
    
    # Create a mapping from match_key to average LMP
    lmp_map = dict(zip(lmp_data['match_key'], lmp_data['avg_lmp']))
    
    # Update the Price column in target data
    price_col = "Price ($/MWh)"
    if price_col in target_data.columns:
        logging.info(f"Updating {price_col} with average LMP for {iso_name}")
        for idx, row in target_data.iterrows():
            match_key = row['match_key']
            if match_key in lmp_map:
                target_data.at[idx, price_col] = lmp_map[match_key]
    else:
        logging.warning(f"Price column '{price_col}' not found in target data for {iso_name}")
    
    # Clean up the temporary match_key column
    if 'match_key' in target_data.columns:
        target_data.drop('match_key', axis=1, inplace=True)
    
    return target_data

if __name__ == "__main__":
    logging.info("Starting LMP data transfer process")
    process_lmp_data()
    logging.info("LMP data transfer process completed")